In [1]:
import pandas as pd
import numpy as np

# Your 3 "Pillars" of performance
FILES = {
    'titan': 'submission_titan_blend.csv',         # 8.54849 (The Leader)
    'push': 'submission_final_push.csv',           # 8.54853
    'ensemble': 'submission_ensemble_8.584346.csv' # 8.54853
}

def main():
    print("Loading the 3 Pillars...")
    dfs = {k: pd.read_csv(v)['exam_score'].values for k, v in FILES.items()}
    
    # We create a stack of the 3 models
    # Shape will be (Rows, 3)
    data_stack = np.stack([dfs['titan'], dfs['push'], dfs['ensemble']], axis=1)
    
    # 1. THE WEIGHTED MEDIAN
    # We repeat the 'titan' values to give them more "votes" in the median calculation
    # This ensures the median stays close to your best score but allows the others 
    # to 'veto' values that look like noise.
    # Logic: [Titan, Titan, Titan, Push, Ensemble] -> Take Median
    weighted_stack = np.stack([
        dfs['titan'], 
        dfs['titan'], 
        dfs['titan'], 
        dfs['push'], 
        dfs['ensemble']
    ], axis=1)
    
    final_preds = np.median(weighted_stack, axis=1)
    
    # 2. THE TINY NUDGE (Post-Processing)
    # Sometimes synthetic data has a slight 'rounding' bias. 
    # We apply a micro-clipping to ensure we hit the 19.6 - 100.0 bounds perfectly.
    final_preds = np.clip(final_preds, 19.6, 100.0)
    
    # Save
    sub = pd.read_csv('sample_submission.csv')
    sub['exam_score'] = final_preds
    output_file = 'submission_weighted_median.csv'
    sub.to_csv(output_file, index=False)
    
    print(f"Created {output_file}")
    print("This approach is much more resistant to overfitting than the Gradient Push.")

if __name__ == "__main__":
    main()

Loading the 3 Pillars...
Created submission_weighted_median.csv
This approach is much more resistant to overfitting than the Gradient Push.
